In [10]:
from scipy.optimize import fmin_bfgs, fmin_l_bfgs_b
import csv
import numpy as np
import math
from collections import defaultdict
import operator
import hashlib
from datetime import datetime
import functools

In [11]:
with open('categories.txt', 'rb') as content_file:
    categories = content_file.read().split('\n')
#to make things faster i put the index where every category starts in a dictionary where the key are the product ids and 
#the values are the starting index of the category
catindex = defaultdict(int)
f = open('categories.txt','rb')
for i,l in enumerate(f):
    if len(l)-len(l.strip()) <=1:
        catindex[l.strip()]=i

In [12]:
#takes product id and returns category
def getCategory(pid, categories, catdict, depth):
    index = catdict[pid]
    value = set()
    j = index + 1
    while len(categories[j])-len(categories[j].strip())>1:
                ws = (categories[j].strip()).split(',')
                break
    return str((ws[:depth])[-1].strip())

In [13]:
def getYear(unix):
    time = datetime.utcfromtimestamp(float(unix))
    return time.strftime("%Y")

In [14]:
data = []
with open('100 core subset filtered.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['uid'], row['pid'], float(row['rating']), row['time']))

In [15]:
data[0]

('A1YXHALQFWN45C', '0783885008', 4.0, '1288310400')

In [16]:
def gethexdig(txt):
    c = hashlib.md5(txt.encode())
    return c.hexdigest()

In [17]:
gs= defaultdict(int)
for d in data:
    cat = getCategory(d[1], categories, catindex, 3)
    gs[cat]+=1
gs = set(sorted(gs.iteritems(), key= operator.itemgetter(1), reverse=True)[:50])
ghashed = set()
for g in gs:
    ghashed.add(gethexdig(g[0])) 

In [18]:
itemsuserrated = defaultdict(lambda: list())
useritems = defaultdict(lambda: list())
itemsuserratedwithtime = defaultdict(lambda: list())
for d in data:
    itemsuserrated[d[0]].append((d[1], d[3]))

for u in itemsuserrated:
    items = itemsuserrated[u]
    isorted = sorted(items, key = lambda tup: tup[1])
    itemsuserrated[u] = isorted
    useritems[u] = [i[0] for i in isorted]
    itemsuserratedwithtime[u] = [(i[0], i[1]) for i in isorted]

In [19]:
def same_year(u1,u2):

    d1= datetime.fromtimestamp(float(u1))
    d2= datetime.fromtimestamp(float(u2))
    delta = d1 - d2
#     if delta.days <=365 and delta.days >= 0:
#         print d1,d2, delta
    return delta.days <=365 and delta.days >= 0

In [20]:
def reviews_before_in_same_year(uid, pid):
    pids = []
    reviews = useritems[uid]
    reviewswithtime = itemsuserratedwithtime[uid]
    index = list(reviews).index(pid)
    year = reviewswithtime[index][1]
    for i in range(index-1):
        if same_year(year, reviewswithtime[i][1]):
            pids.append(reviewswithtime[i][0])
    return pids, reviews[index-1]

In [21]:
user = 'A13G1TKIKHGV3F'
item = 'B000N7AJTW'
item = 'B000L4056E'
x, y = reviews_before_in_same_year(user, item)

len(x)

344

# Every tuple in this dataset is as follows:
( user id, product id, rating score, current genre, previous genres ....)
user id: 0
product id: 1
rating score: 2
current genre: 3 
previous genres: 4-end

In [22]:
datag = []
depth = 3
dg = 'anything'
for d in data:
#     print d[0], d[1]
    index = list(useritems[d[0]]).index(d[1])
    cgenre = getCategory(d[1],categories,catindex,depth)
    if gethexdig(cgenre) not in ghashed:
        cgenre = dg
    cats = defaultdict(int)
    pgenre = ''
    if not index==0:
        pids, preview = reviews_before_in_same_year(d[0], d[1])
        for p in pids:
            cat = getCategory(p, categories, catindex, depth)
            if gethexdig(cat) not in ghashed :
                cat = dg
            cats[cat]+=1
        pgenre = getCategory(preview, categories, catindex, depth)
        if gethexdig(pgenre) not in ghashed:
            pgenre = dg
    tup = (d[:3] + (cgenre,))
    if pgenre!='':
        tup = tup + (pgenre, )
    for c in cats:
        tup = tup + (c, cats[c])
    datag.append(tup)

In [23]:
print datag[0]

('A1YXHALQFWN45C', '0783885008', 4.0, 'Historical', 'Food & Wine', 'Europe', 1, 'Food & Wine', 10, 'Memoirs', 1, 'anything', 10, 'Arts & Literature', 1, 'True Accounts', 9, 'Used & Rental Textbooks', 1, 'Contemporary', 1, 'British', 2, 'Writing', 3, 'Books', 7, 'Asia', 1, 'Historical', 3, 'Military', 2)


In [24]:
users = {}
products = {}
us = set()
ps = set()
genres = defaultdict(int)
gs  = set()
i = 0
j = 0
k = 0
for d in datag:
    if d[0] not in us:
        users[d[0]] = i
        i+=1
        us.add(d[0])
    if d[1] not in ps:
        products[d[1]] = j
        j+=1
        ps.add(d[1])
    if d[3] not in gs:
        gs.add(d[3])
        genres[d[3]] = k
        k+=1

In [25]:
gside = len(genres)
gside

51

In [26]:
# datag = list(set(datag))
# # np.random.shuffle(datag)
# dtest = set()
# dvalid= set() #to choose best regualrization parameter and dimninsionality of factor vectors 
# dtestusers = defaultdict(int)
# dvalidusers = defaultdict(int)
# for d in datag:
#     cr = len(itemsuserrated[d[0]])
#     vc = int((cr * 20)/float(100))
#     if dtestusers[d[0]]< vc:
#         dtestusers[d[0]] +=1
#         dtest.add(d)
#     if d not in dtest and dvalidusers[d[0]]<vc:
#         dvalidusers[d[0]]+=1
#         dvalid.add(d)

# dtrain = set()
# for d in datag:
#     if d not in dtest and d not in dvalid:
#         dtrain.add(d)
# dtrain = list(dtrain)
# dtest = list(dtest)

In [27]:
x60 = int((len(datag) * 60) / float(100))
x20 = int((len(datag) * 20) / float(100))
dtrain = datag[:x60]
dvalid = datag[x60: x60+ x20]
end =  x60+ x20
dtest = datag[end:]
print len(dtrain), len(dvalid), len(dtest), len(datag)
print len(dtrain) + len(dvalid) + len(dtest) == len(datag)

40227 13409 13410 67046
True


In [44]:
dtrain[0][0]

'A1YXHALQFWN45C'

In [29]:
us = set()
ps = set()
gsl = set()
gss = set()
for d in datag:
    us.add(d[0])
    ps.add(d[1])
    for i in range(5,len(d),2):
        gsl.add((d[i], d[3]))
    for i in range(4, len(d)):
        gss.add((d[i], d[3]))
        break

In [30]:
print len(gsl), len(gss)

2601 2322


In [31]:
uintrain = set()
pintrain = set()
glintrain = set()
gsintrain = set()
for d in dtrain:
    pintrain.add(d[1])
    uintrain.add(d[0])
    for i in xrange(5, len(d),2):
        glintrain.add((d[i], d[3]))
    for i in range(4, len(d)):
        gsintrain.add((d[i], d[3]))
        break

In [32]:
len(gsintrain)

2077

In [33]:
def getPredictions(params,k, data):
    alphao = params[0]
    betauo = params[1: len(users)+1]
    end = len(users)+1
    betaio = params[end: end+len(products)]
    end = end+len(products)
    bijs = np.array(params[end: end+(gside**2)])
    betaggs = bijs.reshape(gside, gside)
    end = end + (gside**2)
    bijl = np.array(params[end: end+(gside**2)])
    betaggl = bijl.reshape(gside, gside)  
    end = end + (gside**2)
    gs = np.array(params[end:])
    gr = gs.reshape(len(users)+len(products),k)
    gammuo = gr[:len(users),:]
    gammio = gr[len(users):,:]
    predictions = []
    for d in data:
        p = alphao + betauo[users[d[0]]] + betaio[products[d[1]]] + (gammuo[users[d[0]]].T.dot(gammio[products[d[1]]]))
        for i in range(4,len(d)):
            p+= betaggs[genres[d[i]]][genres[d[3]]]
            break
        for i in xrange(5, len(d), 2):
            p+= betaggl[genres[d[i]]][genres[d[3]]] * np.log10(d[i+1])
        predictions.append(p)
    return predictions

In [34]:
def mse(params,k, data):
    s = 0
    predictions = getPredictions(params, k, data)
    for i, d in enumerate(data):
        s+= (predictions[i] - d[2])**2
    s = s
    return s / float(len(data))

In [35]:
def mean(data):
    s = 0
    for d in data:
        s+= d[2]
    return  s/ float(len(data))

In [36]:
def variance(data):
    m = mean(data)
    s = 0
    for d in data:
        s += (m - d[2]) ** 2
    return s/float(len(data))

In [37]:
def rmse(params,k,data):
    predictions = getPredictions(params, k, data)
    s = 0
    for i, d in enumerate(data):
        s+= (predictions[i] - d[2])**2
    return math.sqrt(s / float(len(data)))

In [38]:
def Rsquare_statistic(params,k, data):
    predictions = getPredictions(params,k, data)
    error = mse(params,k, data)
    var = variance(data)
    return error/float(var)

In [39]:
def compute_cost(initial,lam1,lam2,lam3, k, data):
    alpha = initial[0]
    betau = initial[1:len(users) + 1]
    end = len(users)+1
    betai = initial[end:end + len(products)]
    end = end + len(products)
    bijs = np.array(initial[end: end+(gside**2)])
    betaggs = bijs.reshape(gside, gside)
    end = end + (gside**2)
    bijl = np.array(initial[end: end+(gside**2)])
    betaggl = bijl.reshape(gside, gside)  
    end = end + (gside**2)
    gss = np.array(initial[end:])
    gr = gss.reshape(len(users)+len(products),k)
    gammu = gr[:len(users),:]
    gammi = gr[len(users):,:]
    J = 0
    for d in data:
        term = alpha + betau[users[d[0]]] + betai[products[d[1]]] +(gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2]
        for i in range(4, len(d)):
            term += betaggs[genres[d[i]]][genres[d[3]]]
            break
        for i in xrange(5, len(d), 2):
            term +=  betaggl[genres[d[i]]][genres[d[3]]] * np.log10(d[i+1])
        J+= term ** 2
    for u in us:
        J += lam1 * (np.square(betau[users[u]])+ np.sum(np.square(gammu[users[u]])))
    for p in ps:
        J+= lam1 * ( np.square(betai[products[p]]) + np.sum(np.square(gammi[products[p]])))
    for g in gsintrain:
        J+= lam2 * np.square(betaggs[genres[g[0]]][genres[g[1]]])
    for g in glintrain:
        J+= lam3 * np.square(betaggl[genres[g[0]]][genres[g[1]]])
    gr = compute_gradient(alpha, betau, betai, betaggs,betaggl, gammu, gammi,lam1,lam2, lam3, k, data)
#     print J
    return J, gr.flatten()

In [40]:
def compute_gradient(alpha,betau, betai,betaggs,betaggl, gammu, gammi,lam1, lam2,lam3, k, data):
    ga = 0
    gbu = np.zeros(len(users))
    gbi = np.zeros(len(products))
    ggu = np.zeros((len(users),k))
    ggi = np.zeros((len(products),k))
    gbggs = np.zeros((gside, gside))
    gbggl = np.zeros((gside, gside))
    for d in data:
        term = alpha + betau[users[d[0]]] + betai[products[d[1]]] +(gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2]
        for i in range(4, len(d)):
            term += betaggs[genres[d[i]]][genres[d[3]]]
            break
        for i in xrange(5, len(d), 2):
            term +=  betaggl[genres[d[i]]][genres[d[3]]] * np.log10(d[i+1])
        ga+= 2 * term
        gbu[users[d[0]]] += 2 * term
        gbi[products[d[1]]] += 2 * term
        ggu[users[d[0]]] += 2 * gammi[products[d[1]]] * term
        ggi[products[d[1]]] += 2 * gammu[users[d[0]]] * term
        for i in xrange(5, len(d),2):
            gbggl[genres[d[i]]][genres[d[3]]] += 2* term * np.log10(d[i+1])
        for i in range(4, len(d)):
            gbggs[genres[d[i]]][genres[d[3]]] += 2 * term
            break
    for u in us:
        gbu[users[u]] += 2 * lam1 * betau[users[u]]
        ggu[users[u]] += 2 * lam1 * gammu[users[u]]
    for p in ps:
        gbi[products[p]] += 2 * lam1 * betai[products[p]]
        ggi[products[p]] += 2 * lam1 * gammi[products[p]]
    for g in gsintrain:
        gbggs[genres[g[0]]][genres[g[1]]] += 2 * lam2 * betaggs[genres[g[0]]][genres[g[1]]]
    for g in glintrain:
        gbggl[genres[g[0]]][genres[g[1]]] += 2 * lam3 * betaggl[genres[g[0]]][genres[g[1]]]
    return np.array([ga] + list(gbu) + list(gbi) + list(gbggs.flatten()) + list(gbggl.flatten()) + (
            list(ggu.flatten())+ list(ggi.flatten())))

In [41]:
class ConvergedException(Exception):
    pass
def check(xk):
    global result
    global minmse
    global count
    global presult
    global nit 
    
    nit +=1
#     print nit

    cmse = mse(xk,k, dvalid)
    
    if cmse > minmse:
        if nit >= 100:
            count +=1
            if count == 5:
                print 'here'
                result[:] = presult
                raise ConvergedException()
                return

    else:
        minmse = cmse
        presult[:] = xk
        count = 0
#     print cmse, minmse, presult[0]
    return
    

In [42]:
k = 10
s = 0
for d in datag:
    s+= d[2]
alpha = s / float(len(datag))
betau = np.random.normal(0,0.1, len(users))
betai = np.random.normal(0,0.1, len(products))
betaggs = np.random.normal(0,0.1, (gside, gside))
betaggl = np.random.normal(0,0.1, (gside, gside))

for u in users:
    if u not in uintrain:
        betau[users[u]] = 0
for p in products:
    if p not in pintrain:
        betai[products[p]] = 0
for g in gss:
    if g not in gsintrain:
        betaggs[genres[g[0]]][genres[g[1]]] = 0
for g in gsl:
    if g not in glintrain:
        betaggl[genres[g[0]]][genres[g[1]]] = 0
initial = [alpha] + list(betau) + list(betai) + list(betaggs.flatten()) + list(betaggl.flatten())
gammu = np.random.normal(0,0.1, (len(users), k)) 
gammi = np.random.normal(0,0.1, (len(products), k)) 
for u in users:
    if u not in uintrain:
        gammu[users[u]] = 0
for p in products:
    if p not in pintrain:
        gammi[products[p]] = 0
initialg= initial + list(gammu.flatten()) + list(gammi.flatten())

In [43]:
class ConvergedException(Exception):
    pass
def check_updated(xk):
    global result
    global minmse
    global count
    global presult
    global nit 
    
    nit +=1
#     print nit

    cmse = mse(xk,k, dvalid)
    
    if cmse > minmse:
            count +=1
            if count == 30:
                print 'here'
                result[:] = presult
                raise ConvergedException()
                return

    else:
        minmse = cmse
        presult[:] = xk
        count = 0
#     print cmse, minmse, presult[0]
    return
    

In [45]:
class ConvergedException(Exception):
    pass
def check_updated(xk):
    global result
    global minmse
    global count
    global presult
    global nit 
    
    nit +=1
#     print nit

    cmse = mse(xk,k, dvalid)
    
    if cmse > minmse:
            count +=1
            if count == 30:
                print 'here'
                result[:] = presult
                raise ConvergedException()
                return

    else:
        minmse = cmse
        presult[:] = xk
        count = 0
#     print cmse, minmse, presult[0]
    return
    

In [46]:
# lams = [0.001, 0.01, 0.1, 1, 10, 15, 20, 30]
# 10 - 15 - 20
lams1 = [10]
lams2 = [100]
lams3 = [1000]
for l1 in lams1:
    for l2 in lams2:
        for l3 in lams3:
            presult = []
            count = 0
            nit = 0
            minmse = 1000
            result = []
            pms = []
            try:
                pms, c,d = fmin_l_bfgs_b(compute_cost, x0=initialg,args=(l1,l2, l3, k, dtrain, ), disp = 0, callback = check_updated)
                pms = presult
            except ConvergedException:
                pms = result
            print 'converged after: ' + str(nit)
            print "lambda " + '"' + str(l1) +  '": ' +  ' ' + str(mse(pms, k, dvalid)) + ' ' + str(mse(pms,k, dtest))

here
converged after: 111
lambda "  Books, Children's Books, Animals, Bears
":  0.76854390993 0.750970356264


In [36]:
lams1 = [10]
lams2 = [100]
# lams3 = [1000]
lams3 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
with open('Regression baseline -long & short term memory 100 core.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=['lambda1','lambda2','lambda3', 'k','value'])
    writer.writeheader()
    for l1 in lams1:
        for l2 in lams2: 
            for l3 in lams3:
                minmse = mse(initialg,k,dvalid)
                presult = []
                params = []
                nit = 0
                count = 0
                result = []
                try:
                    params, c,d = fmin_l_bfgs_b(compute_cost, x0=initialg,args=(l1, l2 ,l3, k, dtrain, ), disp = 0, callback = check)
                except ConvergedException:
                    params = result
                print 'converged'
                print "lambda "+ '"' +  str(l1) + '" -> "' + str(l2) + '" -> "'+ str(l3)  + ' " : ' + str(mse(params, k, dtest))
                for i, v in enumerate(params):
                    writer.writerow({'lambda1': l1,'lambda2': l2, 'lambda3': l3,  'k': k, 'value': v})   

here
converged
lambda "10" -> "100" -> "0 " : 0.770984129413
here
converged
lambda "10" -> "100" -> "0.0001 " : 0.770126068939
here
converged
lambda "10" -> "100" -> "0.001 " : 0.770333887677
here
converged
lambda "10" -> "100" -> "0.01 " : 0.770402344866
here
converged
lambda "10" -> "100" -> "0.1 " : 0.76946136941
here
converged
lambda "10" -> "100" -> "1 " : 0.768207417156
here
converged
lambda "10" -> "100" -> "10 " : 0.757078647452
here
converged
lambda "10" -> "100" -> "100 " : 0.751502439841
here
converged
lambda "10" -> "100" -> "1000 " : 0.750955153962
here
converged
lambda "10" -> "100" -> "10000 " : 0.75305112606


In [37]:
params = defaultdict(lambda: defaultdict(list))
with open('Regression baseline -long & short term memory 100 core.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        params[(float(row['lambda1']), float(row['lambda2']), float(row['lambda3']))][int(row['k'])].append(float(row['value']))

In [49]:
op = []
k = 10
minCost = 1000000
predictions = []
mselist = []
lamslist = []
for l in params:
    for k in params[l]:
        c= mse(params[l][k], k ,dvalid)
        print l[2], c
        mselist.append(c)
        lamslist.append(l[2])
        if c < minCost:
            minCost = c
            minlambda = l[2]
            mink = k
            op = params[l][k]
minlambda

0.1 0.788575179858
10000.0 0.768969684455
0.01 0.789251674787
0.0001 0.789061612211
1000.0 0.768060043256
1.0 0.786446107487
100.0 0.768878359628
0.001 0.789169192028
0.0 0.789486731594
10.0 0.774922487728


1000.0

In [50]:
with open("Combined Memory Model Optimized Parameters.csv", "wb") as f:
    writer = csv.writer(f)
    for o in op:
        writer.writerow([o])

In [51]:
# op = []
# with open("Combined Memory Model Optimized Parameters.csv", 'rb') as csvfile:
#     reader = csv.reader(csvfile)
#     for row in reader:
#         op.append(float(row[0]))

In [52]:
print "mse: " + str(mse(op,k,dtest))
# print "R Squared Statistic: " + str( mse(op,k, dtest) /float(variance(data)))

mse: 0.750955153962


In [53]:
mse(op,mink, dtrain)

0.5975815276868327

In [94]:
import matplotlib
from pylab import *
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
lams3 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

x = lams3
y = [0.781657069775, 0.7817087462, 0.781379285749, 0.781454718585, 0.781328608447, 0.779214398316, 0.770226633278, 0.766892173615,
     0.768272918469, 18.361955707]
# color = ['blue'] * 7 + ['red'] + ['blue'] * 3
plt.scatter(lams2[7], y[7], color = 'black', s = 100, marker="o")
plt.semilogx(x, y, color = 'red')
# legend = plt.legend(loc='upper right', shadow=True, fontsize='large')
plt.xlabel('Lambda Value For Combined Model 1 Year Memory Pairwise Genre Bias Terms')
plt.ylabel('mse')
plt.title("C")
plt.show()


In [66]:
# with open('Regression baseline -long & short term memory 100 core 2.csv', 'wb') as f:
#     writer = csv.DictWriter(f, fieldnames=['lambda1','lambda2','lambda3', 'k','value'])
#     writer.writeheader()
#     for l1 in params2:
#         for l2 in params2[l1]:
#             for v in params2[l1][l2]:
#                 writer.writerow({'lambda1': 10,'lambda2': l1,'lambda3':l2,  'k': k, 'value': v})   

In [ ]:
# with open('1 year memory log #times - mse vs lambda.csv', 'wb') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=['lambda1','lambda2','mse'])
#     writer.writeheader()
#     for i in range(len(lams2)):
#         writer.writerow({'lambda1': 10, 'lambda2': lams2[i], 'mse': y[i]})

In [72]:
# params2 = defaultdict(lambda: defaultdict(list))
# le = len(initialg)
# lams3 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
# i = 0
# for p in params:
#     for k in params[p]:
#         for c, j in enumerate(params[p][k]):
#             if c%le==0 and c>0:
#                 params2[100][lams3[i]] = params[p][k][c-le: c]
#                 print lams3[i]
#                 i+=1
# print c
# params2[100][lams3[i]] = params[p][k][c-le:c]
    

In [82]:
# print len(params2[100][100]), len(params2[100][1]), len(initialg)

In [62]:
# op = []
# k = 10
# minCost = 1000000
# predictions = []
# mselist = []
# for l1 in params2:
#     for l2 in params2[l1]:
# #         print len(params2[l1][l2])
#         c= mse(params2[l1][l2], k ,dvalid)
#         mselist.append(c)
# #             print l1, k, c
#         if c < minCost:
#             minCost = c
#             minlambda = l2
#             mink = k
#             op = params2[l1][l2]
# minlambda

100

In [67]:
# op = []
# minCost = 1000000
# predictions = []
# mselist = []
# for l in params:
#     for k in params[l]:
#         c= mse(params[l][k], k ,dvalid)
#         mselist.append(c)
# #             print l1, k, c
#         if c < minCost:
#             minCost = c
#             minlambda = l
#             mink = k
#             op = params[l][k]
# mink, minlambda

In [113]:
def computeNumericalGradient(params,grad, lam,k, data):
    e = 1e-4
    numgrads = np.zeros(len(params))
    pp = np.zeros(len(params))
    for i,p in enumerate(params):
        pp[i] = e
        loss1,_ = compute_cost(params-pp, lam, k,data)
        loss2,_ = compute_cost(params+pp, lam,k, data)
#         print loss1, loss2
        numgrads[i] = (loss2 - loss1) / float(2*e)
        pp[i] = 0
        print round(numgrads[i], 3), round(grad[i], 3)
#         print abs(numgrads[i]-grad[i])
    return numgrads

In [157]:
# alpha = s / float(len(data))
# betau = np.random.normal(0,0.1, len(users))
# betai = np.random.normal(0,0.1, len(products))
# betagg = np.random.normal(0,0.1, (gside, gside))
# initial = [alpha] + list(betau) + list(betai) + list(betagg.flatten())
# gammu = np.random.normal(0,0.1, (len(users), k)) 
# gammi = np.random.normal(0,0.1, (len(products), k)) 
# initial += list(gammu.flatten()) + list(gammi.flatten())
# k = 1
# g = compute_gradient(alpha, betau, betai,betagg, gammu, gammi, 0, k, dtrain)
# computeNumericalGradient(initial,g , 0, k , dtrain )
# # gammu

In [160]:
params = op
k = mink
alphao = params[0]
betauo = params[1: len(users)+1]
end = len(users)+1
betaio = params[end: end+len(products)]
end = end+len(products)
bij = np.array(params[end: end+(gside**2)])
betaggo = bij.reshape(gside, gside)
end = end + (gside**2)
gs = np.array(params[end:])
gr = gs.reshape(len(users)+len(products),k)
gammuo = gr[:len(users),:]
gammio = gr[len(users):,:]

In [161]:
def get_genre_from_id(i):
    for g in genres:
        if genres[g] == i:
            return g

In [162]:
ggbias = defaultdict(int)
for i in range(len(betaggo)):
    for j in range(len(betaggo[i])):
        g1 = get_genre_from_id(i)
        g2 = get_genre_from_id(j)
        ggbias[(g1,g2)] = betaggo[i][j]

In [163]:
sorted(ggbias.iteritems(), key=operator.itemgetter(1), reverse=True)

[(('Books', 'Leaders & Notable People'), 0.38112655412547647),
 (('Christian Living', 'Christian Living'), 0.36637428312966785),
 (('Foreign Language Study & Reference', 'Contemporary'), 0.35840186881188968),
 (('Christian Living', 'Contemporary'), 0.35195573217639631),
 (('Professionals & Academics', 'Writing'), 0.34240553771411059),
 (('Arts & Literature', 'Books'), 0.33096903263920557),
 (('United States', 'Contemporary'), 0.31490209633548011),
 (('Europe', 'Memoirs'), 0.29983526009324613),
 (('Food & Wine', 'Asia'), 0.28239674404853121),
 (('Politics & Government', 'Literary'), 0.2813688069723187),
 (('Science Fiction & Fantasy', 'Ethnic & National'), 0.27595083162110895),
 (('Ancient', 'Europe'), 0.27565680253251273),
 (('Literary', 'Social Sciences'), 0.27438216705738616),
 (('Historical', 'Social Sciences'), 0.26996561926192769),
 (('Books', "Children's Books"), 0.26704606487893706),
 (('True Accounts', 'Economics'), 0.25749426064463299),
 (('Writing', 'Ethnic & National'), 0.25